<a href="https://colab.research.google.com/github/AitMahfoud-Rahma/IGDR_Internship/blob/main/check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
def install_packages():
    try:
        import subprocess
        subprocess.run(["pip", "install", "fuzzywuzzy", "unidecode", "pandas", "Levenshtein"], check=True)
        print("Les packages ont été installés avec succès.")
    except Exception as e:
        print(f"Une erreur s'est produite lors de l'installation des packages : {str(e)}")

install_packages()

Les packages ont été installés avec succès.


In [18]:
# This function doesnt work correctly !!!!
import pandas as pd

def read_excel_file(path_db, path_jotform, sheet_name="Form Responses"):
    # Read the database file
    file_db = pd.read_excel(path_db, header=0)

    # Read the jotform file
    file_jotform = pd.read_excel(path_jotform, sheet_name=sheet_name, header=0)

    return file_db, file_jotform

path_db = "/content/table-data-5.xls"
path_jotform = "/content/FICHE_DE_RENSEIGNEMENTS-4.xlsx"
data_db, data_jotform = read_excel_file(path_db, path_jotform)
################## Unused chunk ##################

In [23]:
import pandas as pd

# Read the database file
path_db = "/content/table-data-5.xls"
file_db = pd.read_excel(path_db, header=0)
sheet_name = "Form Responses"

# Read the jotform file
path_jotform = "/content/FICHE_DE_RENSEIGNEMENTS-4.xlsx"
file_jotform = pd.read_excel(path_jotform, sheet_name=sheet_name, header=0)

In [31]:
from unidecode import unidecode
from fuzzywuzzy import fuzz
import pandas as pd

def check_existence_and_similarity(field_name, jotform_value, db_column, log_messages):
    jotform_value_mod = unidecode(str(jotform_value)).lower()
    db_values_lower = db_column.apply(lambda x: unidecode(str(x)) if pd.notna(x) else x).str.lower()

    # Vérifier si la valeur existe dans la base de données
    exists = jotform_value_mod in db_values_lower.values

    if exists:
        message = f"{field_name} ({jotform_value}) existe déjà dans la base de données."
        log_messages.append(message)
        print(message)
    else:
        # Vérifier la similarité entre les valeurs
        similarity_scores = db_values_lower.apply(lambda x: fuzz.ratio(x, jotform_value_mod) if isinstance(x, str) else 0)
        max_similarity_score = similarity_scores.max()

        if max_similarity_score >= 65:  # Threshold similarity
            # Récupérer la valeur exacte dans la base de données
            value_exact_in_database = db_column.loc[similarity_scores.idxmax()]
            message = f"{field_name} ({jotform_value}) existe déjà dans la base de données avec une légère différence dans l'orthographe. Valeur exacte dans la base de données : {value_exact_in_database}."
            log_messages.append(message)
            print(message)
        else:
            message = f"{field_name} ({jotform_value}) n'existe pas dans la base de données."
            log_messages.append(message)
            print(message)

def main_verification(file_jotform, file_db):
    log_messages = []

    if 'Affixe_Original' not in file_jotform.columns:
        # Store original affix in a new temporary column
        file_jotform['Affixe_Original'] = file_jotform['Affixe']

    affixe_without_nan = file_db['Affixe'].dropna()
    unique_values = affixe_without_nan.unique()
    unique_values_mod = affixe_without_nan.apply(lambda x: unidecode(x).replace(' ', '').lower()).unique()
    file_jotform['Affixe'] = file_jotform['Affixe'].dropna().apply(lambda x: unidecode(x).replace(' ', '').lower())

    result = file_jotform['Affixe'].isin(unique_values_mod)

    for i, affixe_jotform in enumerate(file_jotform['Affixe']):
        affixe_jotform_mod = unidecode(affixe_jotform).lower().strip()
        affixe_original = file_jotform['Affixe_Original'][i]

        if affixe_jotform_mod in unique_values_mod:
            index_base_de_donnees = unique_values_mod.tolist().index(affixe_jotform_mod)
            affixe_base_de_donnees = unique_values[index_base_de_donnees]

            message = f"L'affixe : ({affixe_original}) existe déjà dans la base de données sous le nom : ({affixe_base_de_donnees})"
            log_messages.append(message)
            print(message)
        else:
            message = f"L'affixe : ({affixe_original}) n'existe pas dans la base de données."
            log_messages.append(message)
            print(message)

        puce_jotform = file_jotform['Puce'][i]
        if pd.notna(puce_jotform):
            nombre_chiffres_puce = sum(c.isdigit() for c in str(puce_jotform))

            if nombre_chiffres_puce < 15:
                nom_usuel_correspondant = file_jotform['Nom Usuel (animal)'][i]
                message_puce = f"Une puce avec un nombre de chiffres de : {nombre_chiffres_puce} a été détectée.\n Veuillez vérifier le pays d'origine de l'animal avec la Puce : {puce_jotform}. Nom Usuel correspondant : {nom_usuel_correspondant}. Numéro de ligne dans file_jotform : {i + 1}"
                log_messages.append(message_puce)
                print(message_puce)

          # Vet check
        check_existence_and_similarity(
            "Le vétérinaire",
            file_jotform['Nom (vétérinaire)'][i] + ' ' + file_jotform['Prénom (vétérinaire)'][i],
            file_db[['Vétérinaire - Nom', 'Vétérinaire - Prénom']].stack(),
            log_messages
        )

        # Proprio check
        check_existence_and_similarity(
            "Le propriétaire",
            file_jotform['Nom (propriétaire)'][i] + ' ' + file_jotform['Prénom (propriétaire)'][i],
            file_db[['Propriétaire - Nom', 'Propriétaire - Prénom']].stack(),
            log_messages
        )

        # Chien check
        check_existence_and_similarity(
            "Le chien",
            file_jotform['Nom Usuel (animal)'][i] ,
            file_db['Nom usuel'],
            log_messages
        )



        with open('output_file.log', 'w') as log_file:
            for message in log_messages:
                log_file.write(message + '\n')



if __name__ == "__main__":
    main_verification(file_jotform, file_db)



L'affixe : (Ferme d'or) n'existe pas dans la base de données.
Le vétérinaire (Labdi Aylene) n'existe pas dans la base de données.
Le propriétaire (Keev Adams) n'existe pas dans la base de données.
Le chien (Zira) existe déjà dans la base de données avec une légère différence dans l'orthographe. Valeur exacte dans la base de données : Ira.
L'affixe : (Du Pic D'Espade) existe déjà dans la base de données sous le nom : (Du Pic D'Espade)
Le vétérinaire (Moner jean) existe déjà dans la base de données avec une légère différence dans l'orthographe. Valeur exacte dans la base de données : Monjean.
Le propriétaire (Gainche Norbert) existe déjà dans la base de données.
Le chien (Nouchka) existe déjà dans la base de données.
L'affixe : (Nouveau afFixe) n'existe pas dans la base de données.
Le vétérinaire (new veterinaire) existe déjà dans la base de données avec une légère différence dans l'orthographe. Valeur exacte dans la base de données : Veterinaire.
Le propriétaire (nouveau proprio) n'exis